In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
new_test = "./new_val_in/new_val_in"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
val_testset  = ArgoverseDataset(data_path=new_test)

In [3]:
batch_sz = 64
batch_sz_test = 1

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    final_inp = []
    final_out = []
    

    for scene in batch:
        old_inp = scene['p_in']
        old_out = scene['p_out']
        
        new_inp = numpy.zeros((60,19,2), dtype=float)
        new_out = numpy.zeros((60,30,2), dtype=float)

        for i in range(60):
            for j in range(18):
                for k in range(2):
                    new_inp[i][j+1][k] = old_inp[i][j+1][k] - old_inp[i][j][k]
                    
                    
       
            for j in range(29):
                for k in range(2):       
                    new_out[i][j+1][k] = old_out[i][j+1][k] - old_out[i][j][k]

                    
        final_inp.append(new_inp)
        final_out.append(new_out)
  
    inp = torch.FloatTensor(final_inp)
    out = torch.FloatTensor(final_out)
    
    
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=0)

In [81]:
def my_collate1(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    inp = []
    agent = []
    
    for scene in batch:
        inp.append(scene['p_in'])
        
        agent_id = scene['agent_id']
    
        for i in range(60):
            if agent_id == scene['track_id'][i][0][0]:
                agent.append(i)
        
        
    
    scene_idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    return scene_idx, inp, agent

test_loader = DataLoader(val_testset, batch_size=batch_sz_test, shuffle=False, collate_fn=my_collate1)

In [48]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class FCModule(nn.Module):
    
    def __init__(self):
        super(FCModule, self).__init__()
        
        self.linear1 = nn.Sequential(
                                    nn.Linear(120*19, 8000), 
                                     nn.ReLU(),         
                                     nn.Linear(8000, 120 * 30)
                                    )

    def forward(self, x):
        x = self.linear1(x)
        
        return x

In [49]:
from tqdm import tqdm_notebook as tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = "cuda:0"
model = FCModule().to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

loss_ema = -1
loss_ema2 = -1

for epoch in range(1):
    for i_batch, sample_batch in enumerate (val_loader):
        inp,out = sample_batch
        inp = inp.cuda()
        out = out.cuda()
        
        
        mixed = torch.cat([inp,out],2).transpose(1,2).reshape(-1,49,120)
        
        y_pred = model(inp.reshape(len(inp),-1)).reshape(-1,60,30,2)
        loss = (torch.mean((y_pred-out)**2))**0.5
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss*0.01
        
        if i_batch%10 == 0:
            print('loss full', epoch, i_batch, loss_ema.item(), loss.item())
       
 # Save the model after every epoch.
    torch.save(model.state_dict(),"usingSequential")

loss full 0 0 0.20690464973449707 0.20690464973449707


KeyboardInterrupt: 

In [82]:
import csv
import pandas as pd
import numpy as np
import gc

gc.collect()

torch.cuda.empty_cache()

device = "cuda:0"
model = FCModule().to(device)
model.load_state_dict(torch.load("usingSequential"))

filename = "output_test.csv"

def test(model, device, test_loader):
    model.eval()
    store = [[]]
    strArr = []
    strArr.append('ID')
    for i in range(60):
        strArr.append('v' + str(i + 1))
    

    with torch.no_grad():
        # Data = input
        for scene_idx, data, agent in test_loader:
            data = data.to(device)
            y_pred = model(data.reshape(len(data),-1)).reshape(-1,60,30,2)
            y_pred = y_pred[-1,0,:,0:2]
            y_pred = y_pred.cpu()
            y_pred_np = y_pred.numpy()
            
            
            
            # 19th timestamp -> 20th timestamp
            # Initialize output shape
            new_out = numpy.zeros((30,2), dtype=float)
            
            
            #print(data.shape)
            
            new_out[0][0] = data[0][agent[0]][18][0] + y_pred_np[0][0]
            new_out[0][1] = data[0][agent[0]][18][1] + y_pred_np[0][1]
                
            for j in range(29):
                for k in range(2):       
                    new_out[j + 1][k] = new_out[j][k] + y_pred_np[j + 1][k]

            
            
            # Convert new out to numpy
            y_pred_np = numpy.array(new_out)
            
            y_pred_np = y_pred_np.flatten()
            
            # Append the result to the csv file.
            y_pred_np = np.insert(y_pred_np, 0, scene_idx[0][0][0][0])
            store.append(y_pred_np)

    with open(filename, 'w',  newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(strArr) 
        csvwriter.writerows(store)

test(model, device, test_loader)

In [83]:
import pandas as pd

file = pd.read_csv('output_test.csv')

file["ID"] = file["ID"].astype('int32')

file.to_csv(path_or_buf="./output_test.csv", header=True, index=False)